In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import json

# Detectron colors
_COLORS = np.array([
    0.000, 0.447, 0.741,
    0.850, 0.325, 0.098,
    0.929, 0.694, 0.125,
    0.494, 0.184, 0.556,
    0.466, 0.674, 0.188
]).astype(np.float32).reshape((-1, 3))

# Random number generator seed
_RNG_SEED = 1

# Fix RNG seeds
random.seed(_RNG_SEED)
np.random.seed(_RNG_SEED)

# Directory where sweep summaries are stored
_DATA_DIR = '../data'

In [ ]:
def load_sweep(sweep_name):
    """Loads a sweep summary."""
    summary_path = os.path.join(_DATA_DIR, '{}.json'.format(sweep_name))
    with open(summary_path, 'r') as f:
        sweep_summary = json.load(f)
    return sweep_summary

In [ ]:
_MIN_P = 0.154
_MIN_F = 0.027

_MAX_P = 0.856
_MAX_F = 0.129

def nw_is_valid_p(job):
    return _MIN_P < job['params'] * 1e-6 and job['params'] * 1e-6 < _MAX_P

def nw_is_valid_f(job):
    return _MIN_F < job['flops'] * 1e-9 and job['flops'] * 1e-9 < _MAX_F

In [ ]:
def compute_norm_ws(cs, num_bins, c_range):
    """Computes normalized EDF weights."""
    hist, edges = np.histogram(cs, bins=num_bins, range=c_range)
    inds = np.digitize(cs, bins=edges) - 1
    assert np.count_nonzero(hist) == num_bins
    return 1 / hist[inds] / num_bins

In [ ]:
# Fixed w and d sweeps
sweeps_fix_w_d = {
    'PNAS': load_sweep('PNAS_fix-w-d'),
    'ENAS': load_sweep('ENAS_fix-w-d'),
    'DARTS': load_sweep('DARTS_fix-w-d')
}

In [ ]:
# NAS sweeps
sweeps_nas = {
    'NASNet': load_sweep('NASNet'),
    'Amoeba': load_sweep('Amoeba'),
    'PNAS': load_sweep('PNAS'),
    'ENAS': load_sweep('ENAS'),
    'DARTS': load_sweep('DARTS')
}

In [ ]:
# Standard DS sweeps
sweeps_std = {
    'NASNet': load_sweep('NASNet'),
    'DARTS': load_sweep('DARTS'),
    'ResNeXt-A': load_sweep('ResNeXt-A'),
    'ResNeXt-B': load_sweep('ResNeXt-B')
}

In [ ]:
print('Figure 8\n')

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

dss = ['PNAS', 'ENAS', 'DARTS']
cms = ['params', 'flops']
cols = [4, 2, 3]

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        sweep = sweeps_fix_w_d[ds]
        if cm == 'params':
            vs = [j['params'] * 1e-6 for j in sweep]
            ax.set_xlim([0, 3])
            ax.set_xlabel('params (M)', fontsize=16)
        if cm == 'flops':
            vs = [j['flops'] * 1e-9 for j in sweep]
            ax.set_xlim([0, 0.5])
            ax.set_xlabel('flops (B)', fontsize=16)
        ax.hist(vs, bins='fd', color=_COLORS[cols[j]], alpha=0.6, label=ds)
    ax.grid(alpha=0.4)
    ax.set_ylabel('number of models', fontsize=16)
    ax.set_ylim([0, 400])
    ax.legend(loc='upper right', prop={'size': 12.5})

plt.tight_layout();

In [ ]:
print('Figure 9\n')

num_bins = 20
dss = ['NASNet', 'Amoeba', 'PNAS', 'ENAS', 'DARTS']
cols = [0, 1, 4, 2, 3]
cms = ['params', 'flops']

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        sweep = sweeps_nas[ds]
        if cm == 'params':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_p(job)])
            ps = np.array([job['params'] * 1e-6 for job in sweep if nw_is_valid_p(job)])
            inds = np.argsort(errs)
            errs, ps = errs[inds], ps[inds]
            ws = compute_norm_ws(ps, num_bins, c_range=(_MIN_P, _MAX_P))
        if cm == 'flops':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_f(job)])
            fs = np.array([job['flops'] * 1e-9 for job in sweep if nw_is_valid_f(job)])
            inds = np.argsort(errs)
            errs, fs = errs[inds], fs[inds]
            ws = compute_norm_ws(fs, num_bins, c_range=(_MIN_F, _MAX_F))
        assert np.isclose(np.sum(ws), 1.0)
        ax.plot(
            errs, np.cumsum(ws),
            color=_COLORS[cols[j]], linewidth=2, alpha=0.8, label=ds
        )
    ax.set_xlabel('error | {}'.format(cm), fontsize=16)
    ax.grid(alpha=0.4)
    ax.set_ylabel('cumulative prob.', fontsize=16)
    ax.set_xlim([5, 12])
    ax.legend(loc='lower right', prop={'size': 13})

plt.tight_layout();

In [ ]:
print('Figure 10\n')

num_bins = 20
dss = ['NASNet', 'Amoeba', 'PNAS', 'ENAS', 'DARTS']
cols = [0, 1, 4, 2, 3]
cms = ['params', 'flops']

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

random.seed(_RNG_SEED)
ks = [2 ** p for p in range(10)]

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        sweep = sweeps_nas[ds]
        if cm == 'params':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_p(job)])
            ps = np.array([job['params'] * 1e-6 for job in sweep if nw_is_valid_p(job)])
            inds = np.argsort(errs)
            errs, ps = errs[inds], ps[inds]
            ws = compute_norm_ws(ps, num_bins, c_range=(_MIN_P, _MAX_P))
        if cm == 'flops':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_f(job)])
            fs = np.array([job['flops'] * 1e-9 for job in sweep if nw_is_valid_f(job)])
            inds = np.argsort(errs)
            errs, fs = errs[inds], fs[inds]
            ws = compute_norm_ws(fs, num_bins, c_range=(_MIN_F, _MAX_F))
        assert np.isclose(np.sum(ws), 1.0)
        cum_ws = np.cumsum(ws)
        # Compute min errs for each k
        k_errs = {}
        for k in ks:
            k_errs[k] = []
            n = len(errs) // k
            for s in range(n):
                s_errs = random.choices(population=errs, cum_weights=cum_ws, k=k)
                k_errs[k].append(np.min(s_errs))
        # Plot means and stds
        ax.scatter(
            np.log2(ks), [np.mean(k_errs[k]) for k in ks],
            color=_COLORS[cols[j]], alpha=0.8, label=ds
        )
        mus = np.array([np.mean(k_errs[k]) for k in ks])
        stds = np.array([np.std(k_errs[k]) for k in ks])
        ax.fill_between(
            np.log2(ks), mus - 2 * stds, mus + 2 * stds,
            color=_COLORS[cols[j]], alpha=0.05
        )
    ax.set_ylabel('error | {}'.format(cm), fontsize=16)
    ax.grid(alpha=0.4)
    ax.set_xlabel('experiment size (log2)', fontsize=16)
    ax.set_ylim([5, 11])
    ax.set_xlim([-0.5, 9.5])
    ax.legend(loc='upper right', prop={'size' : 13})

plt.tight_layout();

In [ ]:
print('Figure 11\n')

num_bins = 20
dss = ['NASNet', 'DARTS', 'ResNeXt-A', 'ResNeXt-B']
cols = [0, 3, 1, 2]
cms = ['params', 'flops']

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        sweep = sweeps_std[ds]
        if cm == 'params':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_p(job)])
            ps = np.array([job['params'] * 1e-6 for job in sweep if nw_is_valid_p(job)])
            inds = np.argsort(errs)
            errs, ps = errs[inds], ps[inds]
            ws = compute_norm_ws(ps, num_bins, c_range=(_MIN_P, _MAX_P))
        if cm == 'flops':
            errs = np.array([job['min_test_top1'] for job in sweep if nw_is_valid_f(job)])
            fs = np.array([job['flops'] * 1e-9 for job in sweep if nw_is_valid_f(job)])
            inds = np.argsort(errs)
            errs, fs = errs[inds], fs[inds]
            ws = compute_norm_ws(fs, num_bins, c_range=(_MIN_F, _MAX_F))
        assert np.isclose(np.sum(ws), 1.0)
        ax.plot(
            errs, np.cumsum(ws),
            color=_COLORS[cols[j]], linewidth=2, alpha=0.8, label=ds
        )
    ax.set_xlabel('error | {}'.format(cm), fontsize=16)
    ax.grid(alpha=0.4)
    ax.set_ylabel('cumulative prob.', fontsize=16)
    ax.set_xlim([5, 12])
    ax.legend(loc='lower right', prop={'size' : 13})

plt.tight_layout();